In [2]:
import copy
import itertools


class TreeNode(object):
    def __init__(self, item=None, times=0, parent=None, firstchild=None, nextsibling=None):
        # 定义了树的节点
        # items是挖掘的内容 times是出现次数 子女兄弟链表建多叉树
        self.item = item
        self.times = times
        self.parent = parent
        self.firstchild = firstchild
        self.nextsibling = nextsibling


class LinkedNode(object):
    # 顶点表中顶点的链表结构
    def __init__(self, item=None, next_node=None):
        # item存放树的节点
        # next_node存放链表的后继
        self.item = item
        self.next_node = next_node


class MultiTree(object):
    # 多叉树
    def __init__(self):
        self.root = TreeNode(item='{}')  # 根节点创建
        self.judge_no_sibling = True

    def insert_pre(self, data_list, dict_L1):  # [([f,c,a,m,p],1),([f,c,a,m,p],1)]
        # dict_L1是顶点表 item frequent null指针
        for item in data_list:
            self.insert(item[0], item[1], dict_L1)

    def insert(self, data_list, data_times, dict_L1):  # [f,a,c,m,p],1,dict_L1
        cur_node = self.root
        for data in data_list:
            if cur_node.firstchild is None:  # 如果为空，则建立节点
                cur_node.firstchild = TreeNode(item=data, times=data_times, parent=cur_node)
                self.add_Vertex_table(cur_node.firstchild, dict_L1)  # 把当前点加入顶点表dict_L1
                cur_node = cur_node.firstchild
            else:
                # 如果不为空，则在兄弟间找，若存在，加times，否则创建
                exist = False
                cur = cur_node.firstchild
                while cur is not None:
                    if cur.item is data:
                        cur.times = cur.times + data_times
                        exist = True
                        cur_node = cur
                        break
                    else:
                        self.judge_no_sibling = False  # 还产生分支，继续递归
                        if cur.nextsibling is not None:
                            cur = cur.nextsibling
                        else:
                            break
                if exist is not True:
                    cur.nextsibling = TreeNode(item=data, times=data_times, parent=cur_node)
                    self.add_Vertex_table(cur.nextsibling, dict_L1)  # 新建的节点
                    cur_node = cur.nextsibling

    def get_single_line(self):
        li = []
        cur_node = self.root
        while (cur_node.firstchild is not None):
            li.append(cur_node.firstchild.item)
            cur_node = cur_node.firstchild
        ret_li = []
        ret_li.extend([[i] for i in li])
        # 把单项放入ret_li

        for i in range(1, len(li)):
            ret_li.extend(itertools.combinations(li, i + 1))
        ret_li = [list(item) for item in ret_li]  # [[],[]]
        return ret_li

    def add_Vertex_table(self, node, dict_L1):  # dict_L1  {'a':(4,null)}
        if dict_L1[node.item][1].next_node is None:
            # 附加头结点的下一个节点
            dict_L1[node.item][1].next_node = LinkedNode(item=node)
        else:  # 不存在，在表尾新建
            cur = cur = dict_L1[node.item][1].next_node
            while cur.next_node is not None:
                cur = cur.next_node
            cur.next_node = LinkedNode(item=node)


class FPgrowth(object):  # 读文件获取data[([a,b,c],1),([b,c,d],1),([a,b,c],1)]
    def read_data(self, pathname,support):
        file = open(pathname, "r")
        data = []
        for line in file.readlines():
            line = line.strip('\n')
            data.append([str(i) for i in line.split()])
        file.close()
        data = [(item, 1) for item in data]
        self.support_num=int(support*len(data)+1)
        return data

    def fp_recursion(self, data):
        tree = FPtree(self.support_num,data)
        tree.L1_generate()
        judge_no_sibling = tree.create_tree()  # 建树并查看有无分支
        if judge_no_sibling is True:
            return tree.tree.get_single_line()
        else:
            vertex_table = tree.vertex_table
            ret_li=[]
            for item in vertex_table:
                # 遍历头表

                li = tree.creat_cond_pattern_base(item[0])
                tmp_ret_li = self.fp_recursion(li)

                ret_li.append([item[0]])
                for i in tmp_ret_li:
                    i.append(item[0])
                    ret_li.append(i)
            return ret_li


class FPtree(object):
    def __init__(self, support_num, data=[]):  # 初始化，新建树，获取data
        self.support_num = support_num
        self.tree = MultiTree()
        self.data = data  # [([a,b,c],1),([b,c,d],1),([a,b,c],1)]
        self.dict_L1 = None
        self.vertex_table = None

    def L1_generate(self):
        mp = {}
        for item in self.data:
            for ele in item[0]:
                if ele not in mp:
                    mp[ele] = item[1]  # item[1]
                else:
                    mp[ele] = int(mp[ele]) + item[1]
        L1 = [(x, (mp[x], LinkedNode())) for x in mp.keys() if int(mp[x]) >=self.support_num]  # 产生L1
        L1 = sorted(L1, key=lambda x: x[1][0], reverse=True)  # 按frequent排序
        # L1为（item，（frequent，Link头结点））

        # 按排序好的L1下标建索引，作为数据库项目排序的依据
        L1_index_dict = dict([(L1[i][0], i) for i in range(len(L1))])
        # L1_item_set为L1单项的set，用来原数据中筛选掉不存在的项
        L1_item_set = set([x[0] for x in L1])
        data2 = []
        for line in self.data:
            #line = list(set(line[0]) & L1_item_set)  # 筛选
            tmp_li=list(set(line[0]) & L1_item_set)

            data2.append((sorted(tmp_li, key=lambda x: L1_index_dict[x]),line[1]))  # 按下标排序
        # data=[[],[],[]]
        self.data = data2  # data变为[([],1),([],1)]
        self.vertex_table = L1  # 有序
        self.dict_L1 = dict(L1)  # dict 乱序

    def create_tree(self):  # 建立FP-tree
        self.tree.insert_pre(self.data, self.dict_L1)
        return self.tree.judge_no_sibling

    def creat_cond_pattern_base(self, item):  # 字母
        li = []
        cur_node = self.dict_L1[item][1]  # 获取链表头结点
        while cur_node.next_node is not None:
            times = cur_node.next_node.item.times
            up_cur = cur_node.next_node.item.parent
            tmp_li = []
            while up_cur.item != '{}':
                tmp_li.append(up_cur.item)  # tmp_li可以是乱序的
                up_cur = up_cur.parent
            li.append((tmp_li, times))
            cur_node=cur_node.next_node
        return li

f=FPgrowth()
print(f.fp_recursion(f.read_data("E://retail.dat",support=0.1)))

[['39'], ['48'], ['39', '48'], ['38'], ['39', '38'], ['32'], ['41'], ['39', '41'], ['48', '41']]
